In [3]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor

from fastapi import FastAPI, UploadFile, File
from io import BytesIO

import pandas as pd
import joblib

In [4]:
df = pd.read_csv('Laptop_price.csv')
X = df.drop(columns=['Price'])
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
num_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object']).columns.tolist()
num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline([    
        ('imputer', SimpleImputer(strategy='most_frequent')), 
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer([
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
])
pipeline = Pipeline([
    ('preprocessor', preprocessor), 
   ('model', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5))
])
pipeline.fit(X_train, y_train)
joblib.dump(pipeline, 'laptop_price_model.pkl')

['laptop_price_model.pkl']

In [6]:
!git init 

Initialized empty Git repository in C:/Users/amir_/OneDrive/Desktop/LR 3/.git/


In [7]:
!git add pipline.ipynb
!git add Laptop_price.csv

In [15]:
!git commit -m "Добавлен ML-пайплайн"

[master 962470a] Добавлен ML-пайплайн
 2 files changed, 470 insertions(+)
 create mode 100644 encript.ipynb
 create mode 100644 laptop_price_model.pkl


In [17]:
!git remote add origin https://github.com/Antelku/lab_pipline.git

error: remote origin already exists.


In [18]:
!git push -u origin master

branch 'master' set up to track 'origin/master'.


Everything up-to-date


In [19]:
%%writefile app.py

app = FastAPI()

# Загрузка обученной модели
model_path = "laptop_price_model.pkl"
model = joblib.load(model_path)

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    content = await file.read()
    df = pd.read_csv(BytesIO(content))
    predictions = model.predict(df)
    return {"predictions": predictions.tolist()}

Writing app.py


In [ ]:
# !nohup uvicorn app:app --host 0.0.0.0 --port 8000 --reload > fastapi.log 2>&1 &


ModuleNotFoundError: No module named 'sklearn'

['C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\\python312.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\\Lib', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0', 'c:\\Users\\amir_\\OneDrive\\Desktop\\LR 3\\.venv', '', 'c:\\Users\\amir_\\OneDrive\\Desktop\\LR 3\\.venv\\Lib\\site-packages', 'c:\\Users\\amir_\\OneDrive\\Desktop\\LR 3\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\amir_\\OneDrive\\Desktop\\LR 3\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\amir_\\OneDrive\\Desktop\\LR 3\\.venv\\Lib\\site-packages\\Pythonwin']
